# 📚 מדריך מקיף: עבודה עם מקורות נתונים בענן ב-Polars

# תקלות בהתחברות!!!

## פרק 11: Working With Common Cloud Sources

---

ברוכים הבאים למדריך המקיף לעבודה עם מקורות נתונים בענן! במדריך זה תלמדו כיצד לקרוא ולכתוב נתונים מפלטפורמות ענן מובילות באמצעות ספריית Polars.

### 📋 תוכן עניינים

1. [אמזון S3 (Amazon S3)](#amazon-s3)
   - [קריאה מ-S3](#reading-from-s3)
   - [כתיבה ל-S3](#writing-to-s3)
   - [קריאה עם אימות](#reading-with-authentication)
   - [עבודה עם PyArrow Dataset](#working-with-pyarrow)

2. [Azure Blob Storage](#azure-blob-storage)
   - [קריאה מ-Azure](#reading-from-azure)
   - [כתיבה ל-Azure](#writing-to-azure)
   - [עבודה עם משתני סביבה](#azure-environment-variables)
   - [Azure Data Lake Storage](#azure-data-lake)

3. [Google Cloud Storage (GCS)](#google-cloud-storage)
   - [קריאה מ-GCS](#reading-from-gcs)
   - [אימות עם Service Account](#gcs-authentication)
   - [כתיבה ל-GCS](#writing-to-gcs)

4. [BigQuery](#bigquery)
   - [קריאה מ-BigQuery](#reading-from-bigquery)
   - [כתיבה ל-BigQuery](#writing-to-bigquery)

5. [Snowflake](#snowflake)
   - [חיבור ל-Snowflake](#connecting-to-snowflake)
   - [קריאת נתונים](#reading-from-snowflake)
   - [שיטות חיבור נוספות](#alternative-snowflake-methods)

6. [תרגילים מעשיים](#exercises)
7. [סיכום ומשאבים נוספים](#summary)

---

## 🎯 מטרות הלמידה

בסיום מדריך זה תדעו:

✅ כיצד להתחבר למקורות נתונים בענן (S3, Azure, GCS, BigQuery, Snowflake)

✅ איך לקרוא ולכתוב קבצים בפורמטים שונים (CSV, Parquet) מהענן

✅ כיצד לנהל אימות והרשאות בצורה מאובטחת

✅ דרכים יעילות לעבוד עם נתונים גדולים בענן

✅ שילוב Polars עם כלי ענן נפוצים

---

## 📦 התקנת חבילות נדרשות

לפני שמתחילים, נוודא שכל החבילות הנדרשות מותקנות:

In [ ]:
# התקנת חבילות נדרשות (הסר את ה-# להרצה)
# !pip install polars
# !pip install s3fs              # עבור Amazon S3
# !pip install adlfs             # עבור Azure Blob Storage
# !pip install gcsfs             # עבור Google Cloud Storage
# !pip install google-cloud-bigquery
# !pip install pyarrow
# !pip install snowflake-connector-python
# !pip install connectorx        # עבור חיבור למסדי נתונים
# !pip install adbc-driver-snowflake  # עבור Snowflake

---

<a id="amazon-s3"></a>
# 1️⃣ Amazon S3 (Simple Storage Service)

## 🔍 מבוא

**Amazon S3** הוא שירות אחסון אובייקטים של AWS המאפשר אחסון והחזרת כמויות גדולות של נתונים. 

### למה להשתמש ב-S3?
- 🌐 **זמינות גבוהה** - 99.999999999% (11 תשיעיות)
- 💰 **עלות נמוכה** - תשלום לפי שימוש
- 📈 **מדרגיות** - תומך בכמויות נתונים בלתי מוגבלות
- 🔐 **אבטחה** - שכבות אבטחה מתקדמות

---

<a id="reading-from-s3"></a>
## 📖 קריאה מ-S3 - השיטה הפשוטה

הדרך הכי פשוטה לקרוא קובץ מ-S3 היא באמצעות הפונקציה `pl.read_csv()` עם נתיב S3.

In [ ]:
import polars as pl

# הדפסת גרסת Polars
print(f"🐻‍❄️ Polars version: {pl.__version__}")

### 💡 דוגמה: קריאת קובץ CSV מ-S3

In [ ]:
# שלב 1: הגדרת נתיב הקובץ ב-S3
# פורמט: s3://bucket-name/path/to/file.csv
s3_file_path = 's3://polars-cookbook-demo-yk/titanic_dataset.csv'

# שלב 2: קריאת הקובץ
df = pl.read_csv(s3_file_path)

# שלב 3: הצגת 5 השורות הראשונות
print("✅ הנתונים נקראו בהצלחה מ-S3!")
print(f"📊 מספר שורות: {df.shape[0]:,}")
print(f"📋 מספר עמודות: {df.shape[1]}")
df.head()

### 🎓 הסבר מפורט:

1. **נתיב S3**: `s3://bucket-name/file.csv`
   - `s3://` - פרוטוקול S3
   - `bucket-name` - שם ה-bucket (כמו תיקייה ראשית)
   - `file.csv` - נתיב הקובץ בתוך ה-bucket

2. **קריאה אוטומטית**: Polars מזהה את הפרוטוקול `s3://` ומטפל באופן אוטומטי בחיבור

3. **ללא אימות**: דוגמה זו עובדת עם bucket פומבי (public)

---

<a id="writing-to-s3"></a>
## 📝 כתיבה ל-S3 - שמירת קבצים

כדי לכתוב קבצים ל-S3, נשתמש בספריית `s3fs` שמספקת ממשק קבצים ל-S3.

In [ ]:
import s3fs

# שלב 1: יצירת מופע של S3FileSystem
fs = s3fs.S3FileSystem()

# שלב 2: הגדרת נתיב היעד ב-S3
s3_parquet_file_path = 's3://polars-cookbook-demo-yk/titanic_dataset.parquet'

# שלב 3: פתיחת הקובץ במצב כתיבה וכתיבת ה-DataFrame
with fs.open(s3_parquet_file_path, mode='wb') as f:
    df.write_parquet(f)

print("✅ הקובץ נשמר בהצלחה ל-S3 בפורמט Parquet!")

### 🎓 הסבר מפורט:

1. **S3FileSystem**: מחלקה המספקת ממשק דמוי-קובץ לעבודה עם S3

2. **Context Manager (`with`)**: מבטיח סגירה נכונה של החיבור

3. **מצב כתיבה (`'wb'`)**:
   - `w` = write (כתיבה)
   - `b` = binary (בינארי, נדרש ל-Parquet)

4. **פורמט Parquet**:
   - ✅ דחוס ויעיל (50-80% פחות מקום)
   - ✅ קריאה מהירה יותר
   - ✅ שומר טיפוסי נתונים

### 💡 טיפ חשוב:
> **Parquet vs CSV**: השתמשו ב-Parquet לנתונים גדולים ו-CSV רק לקבצים קטנים או לשיתוף אנושי.

---

<a id="reading-with-authentication"></a>
## 🔐 קריאה עם אימות (Authentication)

עבור buckets פרטיים, נדרש אימות באמצעות AWS credentials.

### שיטה 1: שימוש ב-storage_options

In [ ]:
# הגדרת פרמטרי אימות
storage_options = {
    'aws_access_key_id': 'YOUR_ACCESS_KEY_ID',          # מפתח גישה
    'aws_secret_access_key': 'YOUR_SECRET_ACCESS_KEY',  # מפתח סודי
    'aws_region': 'us-east-1'                            # אזור AWS
}

# קריאה עם Lazy evaluation
lf = pl.scan_parquet(s3_parquet_file_path, storage_options=storage_options)

# הרצת החישוב
result = lf.head().collect()
print("✅ נתונים נקראו בהצלחה עם אימות!")
result

### 🎓 הסבר על Lazy Evaluation:

**`scan_parquet()`** מחזיר `LazyFrame` במקום `DataFrame`:

| תכונה | DataFrame | LazyFrame |
|-------|-----------|------------|
| ⚡ ביצוע | מיידי | דחוי (Lazy) |
| 💾 זיכרון | טוען הכל | טוען לפי צורך |
| 🎯 אופטימיזציה | בסיסית | מתקדמת |
| 📊 מתי להשתמש | נתונים קטנים | נתונים גדולים |

---

### שיטה 2: כתיבה עם אימות

In [ ]:
# יצירת S3FileSystem עם credentials
fs = s3fs.S3FileSystem(
    key='YOUR_ACCESS_KEY_ID',
    secret='YOUR_SECRET_ACCESS_KEY'
)

# כתיבה מאומתת
with fs.open(s3_parquet_file_path, mode='wb') as f:
    df.write_parquet(f)

print("✅ הקובץ נשמר ל-S3 פרטי בהצלחה!")

### ⚠️ אזהרת אבטחה:

> **לעולם אל תשמרו credentials בקוד!**
> 
> דרכים מאובטחות:
> 1. משתני סביבה (Environment Variables)
> 2. AWS IAM Roles
> 3. קבצי קונפיגורציה מוצפנים
> 4. AWS Secrets Manager

---

<a id="working-with-pyarrow"></a>
## 🏹 עבודה עם PyArrow Dataset

PyArrow מאפשר קריאה יעילה עם סינון לפני הטעינה.

In [ ]:
import pyarrow.dataset as ds

# שלב 1: יצירת Dataset מ-S3
dataset = ds.dataset(
    s3_parquet_file_path, 
    format='parquet',      # פורמט הקובץ
    filesystem=fs          # מערכת הקבצים (S3)
)

# שלב 2: סינון מתקדם לפני קריאה
df_filtered = (
    pl.scan_pyarrow_dataset(dataset)
    .filter(pl.col('Age') <= 30)  # רק נוסעים עד גיל 30
    .collect()                     # הרצת החישוב
)

print(f"📊 סונן ל-{df_filtered.shape[0]:,} שורות (גיל <= 30)")
df_filtered.head()

### 🎓 יתרונות PyArrow Dataset:

1. **Predicate Pushdown** 🚀
   - הסינון קורה ב-S3 לפני הטעינה
   - חוסך זמן ורוחב פס

2. **Partition Pruning** 📁
   - קורא רק partitions רלוונטיים
   - אידיאלי לנתונים מחולקים

3. **Lazy Evaluation** ⚡
   - מחשב רק מה שנדרש
   - אופטימיזציה אוטומטית

---

## 📊 תרגיל 1: עבודה עם S3

### 🎯 משימה:
קרא את dataset Titanic מ-S3, סנן נוסעים שנשרדו (Survived=1) ובני 18-40, והצג את שכיחות המין.

In [ ]:
# התחל כאן - מלא את הקוד

# שלב 1: קרא את הנתונים מ-S3
s3_path = 's3://polars-cookbook-demo-yk/titanic_dataset.csv'
# df = ...

# שלב 2: סנן לפי התנאים
# df_survivors = df.filter(...)

# שלב 3: חשב שכיחות לפי מין
# result = df_survivors...

# הדפס תוצאה
# result

### 💡 פתרון:

In [ ]:
# פתרון מוסתר - לחץ להרצה

df = pl.read_csv('s3://polars-cookbook-demo-yk/titanic_dataset.csv')

result = (
    df
    .filter(
        (pl.col('Survived') == 1) &
        (pl.col('Age') >= 18) &
        (pl.col('Age') <= 40)
    )
    .group_by('Sex')
    .agg([
        pl.len().alias('Count'),
        pl.col('Age').mean().alias('Avg_Age')
    ])
    .sort('Count', descending=True)
)

print("✅ ניתוח הושלם!")
result

---

<a id="azure-blob-storage"></a>
# 2️⃣ Azure Blob Storage

## 🔍 מבוא

**Azure Blob Storage** הוא שירות אחסון אובייקטים של Microsoft Azure, דומה ל-S3 של AWS.

### למה להשתמש ב-Azure Blob?
- ☁️ **אינטגרציה עם Azure** - חלק מהאקוסיסטם של Microsoft
- 💼 **תמיכה ארגונית** - מתאים לסביבות Enterprise
- 🔄 **רמות אחסון** - Hot, Cool, Archive
- 🌍 **זמינות גלובלית** - data centers ברחבי העולם

---

<a id="reading-from-azure"></a>
## 📖 קריאה מ-Azure Blob Storage

In [ ]:
# הגדרת פרמטרי אימות לAzure
storage_options = {
    'account_name': 'YOUR_ACCOUNT_NAME',      # שם חשבון האחסון
    'access_key': 'YOUR_ACCOUNT_KEY',         # מפתח גישה
    'client_id': 'YOUR_CLIENT_ID',            # (אופציונלי) עבור Service Principal
    'client_secret': 'YOUR_CLIENT_SECRET',    # (אופציונלי)
    'tenant_id': 'YOUR_TENANT_ID'             # (אופציונלי)
}

# נתיב Blob
# פורמט: az://container-name/path/to/file
blob_csv_file_path = 'az://demo/titanic_dataset.csv'

# קריאת הקובץ
df = pl.read_csv(blob_csv_file_path, storage_options=storage_options)

print("✅ הנתונים נקראו בהצלחה מ-Azure!")
df.head()

### 🎓 הסבר על מבנה הנתיב:

```
az://container-name/path/to/file.csv
│   │                │
│   │                └─ נתיב הקובץ בcontainer
│   └─ שם ה-container (דומה ל-bucket ב-S3)
└─ פרוטוקול Azure
```

---

<a id="writing-to-azure"></a>
## 📝 כתיבה ל-Azure Blob Storage

In [ ]:
import adlfs  # Azure DataLake File System

# יצירת מופע של AzureBlobFileSystem
fs = adlfs.AzureBlobFileSystem(
    account_name='YOUR_ACCOUNT_NAME',
    account_key='YOUR_ACCOUNT_KEY'
)

# נתיב היעד
blob_parquet_file_path = 'az://demo/titanic_dataset.parquet'

# כתיבה
with fs.open(blob_parquet_file_path, mode='wb') as f:
    df.write_parquet(f)

print("✅ הקובץ נשמר ל-Azure Blob בהצלחה!")

### 💡 טיפים לעבודה עם Azure:

1. **Container Names**: חייבים להיות באותיות קטנות
2. **Access Tiers**: בחרו את הרמה המתאימה (Hot/Cool/Archive)
3. **SAS Tokens**: אלטרנטיבה מאובטחת ל-account keys

---

## 🔎 קריאה עם Scan (Lazy)

In [ ]:
# קריאה עצלה (Lazy) עם אפשרות לאופטימיזציה
lf = pl.scan_parquet(blob_parquet_file_path, storage_options=storage_options)

# ביצוע פעולות ללא טעינת כל הנתונים
result = lf.head().collect()

print("✅ נתונים נקראו בצורה מאומתת!")
result

## 🏹 שימוש ב-PyArrow Dataset עם Azure

In [ ]:
import pyarrow.dataset as ds

# יצירת dataset
dataset = ds.dataset(
    blob_parquet_file_path,
    filesystem=fs,
    format='parquet'
)

# קריאה עם סינון
df = (
    pl.scan_pyarrow_dataset(dataset)
    .filter(pl.col('Age') <= 30)
    .collect()
)

print(f"📊 נמצאו {df.shape[0]:,} שורות עם גיל <= 30")
df.head()

<a id="azure-environment-variables"></a>
## 🌍 עבודה עם משתני סביבה

דרך מאובטחת יותר לאחסן credentials:

In [ ]:
import os

# הגדרת משתני סביבה
os.environ['AZURE_STORAGE_ACCOUNT_NAME'] = 'YOUR_ACCOUNT_NAME'
os.environ['AZURE_STORAGE_ACCOUNT_KEY'] = 'YOUR_ACCOUNT_KEY'

# כעת Polars יקרא אוטומטית מהמשתנים
df = pl.read_csv(blob_csv_file_path, storage_options=storage_options)

df.head()

### ⚙️ הגדרת משתני סביבה במערכת:

**Windows (PowerShell):**
```powershell
$env:AZURE_STORAGE_ACCOUNT_NAME="your_account"
$env:AZURE_STORAGE_ACCOUNT_KEY="your_key"
```

**Linux/Mac:**
```bash
export AZURE_STORAGE_ACCOUNT_NAME="your_account"
export AZURE_STORAGE_ACCOUNT_KEY="your_key"
```

---

<a id="azure-data-lake"></a>
## 🏞️ Azure Data Lake Storage (ADLS)

ADLS הוא שירות אחסון מתקדם יותר, אופטימלי לביג דאטה.

In [ ]:
# החלפת account name ל-ADLS
storage_options['account_name'] = 'YOUR_ADLS_ACCOUNT_NAME'
storage_options['account_key'] = 'YOUR_ADLS_ACCOUNT_KEY'

# קריאה מADLS (אותו API כמו Blob)
df = pl.read_csv(blob_csv_file_path, storage_options=storage_options)

print("✅ נתונים נקראו מ-ADLS!")
df.head()

### 📊 Blob Storage vs ADLS:

| תכונה | Blob Storage | ADLS Gen2 |
|--------|--------------|------------|
| 🎯 שימוש | קבצים כלליים | ביג דאטה |
| 📁 היררכיה | שטוחה | עצית |
| 🔐 אבטחה | בסיסית | מתקדמת (ACL) |
| 💰 מחיר | נמוך | גבוה יותר |
| ⚡ ביצועים | טוב | מעולה |

---

## 📊 תרגיל 2: עבודה עם Azure

### 🎯 משימה:
צור DataFrame חדש עם רק הנוסעים ממחלקה ראשונה (Pclass=1), שמור אותו ל-Azure Blob בפורמט Parquet.

In [ ]:
# התחל כאן - מלא את הקוד

# שלב 1: סנן מחלקה ראשונה
# first_class = ...

# שלב 2: שמור ל-Azure
# output_path = 'az://demo/first_class_passengers.parquet'
# ...

# שלב 3: קרא חזרה ווודא
# verification = ...


---

<a id="google-cloud-storage"></a>
# 3️⃣ Google Cloud Storage (GCS)

## 🔍 מבוא

**Google Cloud Storage** הוא שירות אחסון אובייקטים של Google Cloud Platform.

### למה להשתמש ב-GCS?
- 🚀 **ביצועים גבוהים** - רשת הגלובלית של Google
- 🔄 **אינטגרציה עם GCP** - BigQuery, Dataflow, וכו'
- 💎 **Storage Classes** - Standard, Nearline, Coldline, Archive
- 🔐 **אבטחה** - הצפנה אוטומטית

---

<a id="reading-from-gcs"></a>
## 📖 קריאה מ-GCS - גישה פומבית

In [ ]:
# נתיב GCS
# פורמט: gs://bucket-name/path/to/file
gcs_csv_file_path = 'gs://polars_cookbook_demo_yk/titanic_dataset.csv'

# קריאה פשוטה מbucket פומבי
df = pl.read_csv(gcs_csv_file_path)

print("✅ הנתונים נקראו מ-GCS!")
print(f"📊 {df.shape[0]:,} שורות × {df.shape[1]} עמודות")
df.head()

### 🎓 מבנה נתיב GCS:

```
gs://bucket-name/folder/file.csv
│   │            │      │
│   │            │      └─ שם הקובץ
│   │            └─ תיקיות (אופציונלי)
│   └─ שם ה-bucket
└─ פרוטוקול GCS
```

---

<a id="gcs-authentication"></a>
## 🔐 אימות עם Service Account

עבור buckets פרטיים, נשתמש ב-Service Account credentials.

### שיטה 1: קובץ JSON

In [ ]:
# נתיב לקובץ credentials (JSON)
credentials_file_path = 'YOUR_FILE_NAME.json'

# העברת הנתיב כ-token
storage_options = {'token': credentials_file_path}

df = pl.read_csv(gcs_csv_file_path, storage_options=storage_options)

print("✅ קריאה מאומתת בהצלחה!")
df.head()

### 📝 איך ליצור Service Account?

1. עבור ל-[Google Cloud Console](https://console.cloud.google.com/)
2. IAM & Admin → Service Accounts
3. Create Service Account
4. הורד את קובץ ה-JSON
5. שמור במקום מאובטח

---

### שיטה 2: טעינת credentials כ-dictionary

In [ ]:
import json

def read_json_into_dict(file_path):
    """טוען קובץ JSON ומחזיר dictionary"""
    try:
        with open(file_path) as file:
            return json.load(file)
    except Exception as err:
        print(f"❌ שגיאה בקריאת הקובץ: {err}")
        raise err

# טעינת ה-credentials
storage_options = read_json_into_dict(credentials_file_path)

# שימוש
df = pl.read_csv(gcs_csv_file_path, storage_options=storage_options)

print("✅ נתונים נטענו עם credentials dictionary!")
df.head()

<a id="writing-to-gcs"></a>
## 📝 כתיבה ל-GCS

In [ ]:
import gcsfs

# יצירת GCSFileSystem
fs = gcsfs.GCSFileSystem()

# נתיב יעד
gcs_parquet_file_path = 'gs://polars_cookbook_demo_yk/titanic_dataset.parquet'

# כתיבה
with fs.open(gcs_parquet_file_path, mode='wb') as f:
    df.write_parquet(f)

print("✅ הקובץ נשמר ל-GCS בפורמט Parquet!")

### כתיבה עם אימות:

In [ ]:
# יצירת filesystem עם credentials
fs = gcsfs.GCSFileSystem(token=credentials_file_path)

with fs.open(gcs_parquet_file_path, mode='wb') as f:
    df.write_parquet(f)

print("✅ כתיבה מאומתת הושלמה!")

## 🏹 PyArrow Dataset עם GCS

In [ ]:
import pyarrow.dataset as ds

# יצירת dataset
dataset = ds.dataset(
    gcs_parquet_file_path,
    filesystem=fs,
    format='parquet'
)

# קריאה עם סינון יעיל
df = (
    pl.scan_pyarrow_dataset(dataset)
    .filter(pl.col('Age') <= 30)
    .collect()
)

print(f"🎯 סונן ל-{df.shape[0]:,} נוסעים צעירים")
df.head()

### 💡 טיפים ל-GCS:

1. **Bucket Naming**: 
   - שמות ייחודיים globally
   - רק אותיות קטנות, מספרים, מקפים

2. **Storage Classes**:
   - Standard: גישה תכופה
   - Nearline: פעם בחודש
   - Coldline: פעם ברבעון
   - Archive: פעם בשנה

3. **Lifecycle Policies**: העבר קבצים ישנים אוטומטית למחלקות זולות יותר

---

## 📊 תרגיל 3: עבודה עם GCS

### 🎯 משימה:
1. קרא נתונים מ-GCS
2. חשב את שיעור ההישרדות לפי מחלקה (Pclass)
3. שמור את התוצאה ל-GCS כ-CSV חדש

In [ ]:
# התחל כאן

# שלב 1: קרא נתונים
# ...

# שלב 2: חשב שיעור הישרדות
# survival_rate = ...

# שלב 3: שמור תוצאה
# output_path = 'gs://your-bucket/survival_by_class.csv'
# ...

---

<a id="bigquery"></a>
# 4️⃣ BigQuery - מסד נתונים ענן של Google

## 🔍 מבוא

**BigQuery** הוא מסד נתונים serverless ומאסיבי במיוחד של Google.

### למה BigQuery?
- ⚡ **מהירות** - שאילתות על טרה-בייטים בשניות
- 📊 **SQL** - שפת שאילתות מוכרת
- 💰 **תמחור** - תשלום לפי שאילתות
- 🔄 **אינטגרציה** - עם כל שירותי GCP

---

<a id="reading-from-bigquery"></a>
## 📖 קריאה מ-BigQuery

### שיטה 1: ConnectorX (מומלץ)

In [ ]:
# פרטי החיבור
project = 'sandbox-366819'
dataset = 'polars_cookbook_demo_yk'
table = 'titanic_dataset'

# שאילתת SQL
query = f'''
    SELECT *
    FROM `{project}.{dataset}.{table}`
    LIMIT 1000
'''

# נתיב ל-credentials
credentials_file_path = 'YOUR_CREDENTIALS_FILE_PATH.json'

# בניית URI
uri = f'bigquery://{credentials_file_path}'

# קריאה
df = pl.read_database_uri(
    query,
    uri,
    engine='connectorx'  # מנוע חיבור מהיר
)

print("✅ נתונים נקראו מ-BigQuery!")
print(f"📊 {df.shape[0]:,} שורות נטענו")
df.head()

### 🎓 הסבר על הפורמט:

```sql
SELECT *
FROM `project.dataset.table`
      │       │       │
      │       │       └─ שם הטבלה
      │       └─ שם ה-dataset
      └─ ID הפרויקט
```

**שימו לב**: שימוש ב-backticks (\`) סביב השם המלא!

---

### שיטה 2: BigQuery Client

In [ ]:
from google.cloud import bigquery

# יצירת client
client = bigquery.Client.from_service_account_json(credentials_file_path)

# הרצת שאילתה
query_job = client.query(query)
rows = query_job.result()

# המרה ל-Polars דרך Arrow
df = pl.from_arrow(rows.to_arrow())

print("✅ נתונים נטענו דרך BigQuery Client!")
df.head()

### 💡 יתרונות Arrow:

- **Zero-copy** - ללא העתקת זיכרון
- **מהיר** - 10-100x מהיר מJSON
- **טיפוסים** - שומר את כל המידע על הנתונים

---

### שיטה 3: read_database

In [ ]:
# שימוש ב-connection object
# (client כבר הוגדר מעלה)

df = pl.read_database(query, connection=client)

print("✅ קריאה ישירה מהחיבור!")
df.head()

<a id="writing-to-bigquery"></a>
## 📝 כתיבה ל-BigQuery

In [ ]:
import io

# טבלת יעד
destination = f'{project}.{dataset}.titanic_dataset_v2'

# המרה ל-CSV בזיכרון
with io.BytesIO() as stream:
    # כתיבה לזיכרון
    df.write_csv(stream)
    
    # חזרה להתחלה
    stream.seek(0)
    
    # העלאה ל-BigQuery
    job = client.load_table_from_file(
        stream,
        destination=destination,
        project=project,
        job_config=bigquery.LoadJobConfig(
            autodetect=True,  # זיהוי אוטומטי של סכמה
            source_format=bigquery.SourceFormat.CSV,
        ),
    )
    
# המתנה לסיום
job.result()

print(f"✅ {df.shape[0]:,} שורות נשמרו ל-BigQuery!")
print(f"📍 טבלה: {destination}")

### 🎓 הסבר על התהליך:

1. **BytesIO**: "קובץ" בזיכרון (לא כותב לדיסק)
2. **seek(0)**: חזרה להתחלת ה-stream
3. **autodetect**: BigQuery מזהה את טיפוסי העמודות
4. **job.result()**: ממתין לסיום העלאה

---

## 💰 טיפים לחיסכון ב-BigQuery:

1. **השתמשו ב-LIMIT**: אל תשלפו יותר ממה שצריך
   ```sql
   SELECT * FROM table LIMIT 1000
   ```

2. **בחרו עמודות ספציפיות**: 
   ```sql
   SELECT name, age FROM table  -- ✅ טוב
   SELECT * FROM table          -- ❌ יקר
   ```

3. **סננו מוקדם**: השתמשו ב-WHERE
   ```sql
   SELECT * FROM table WHERE date >= '2024-01-01'
   ```

4. **Partitioned Tables**: חלקו לפי תאריך

---

## 📊 תרגיל 4: BigQuery

### 🎯 משימה:
כתוב שאילתה שמחזירה את 10 הכרטיסים היקרים ביותר, קרא ל-Polars, והצג.

In [ ]:
# כתוב שאילתה
expensive_tickets_query = """
-- השלם את השאילתה
SELECT 
    ...
"""

# קרא נתונים
# ...

# הצג תוצאות
# ...

---

<a id="snowflake"></a>
# 5️⃣ Snowflake - מסד הנתונים הענני

## 🔍 מבוא

**Snowflake** הוא פלטפורמת data warehouse מודרנית המבוססת כולה על הענן.

### למה Snowflake?
- ❄️ **ארכיטקטורה ייחודית** - הפרדה בין חישוב ואחסון
- 🔄 **שיתוף נתונים** - Data Sharing בין ארגונים
- 📊 **ביצועים** - אופטימיזציה אוטומטית
- 💼 **Enterprise** - פתרון לארגונים גדולים

---

<a id="connecting-to-snowflake"></a>
## 🔌 חיבור ל-Snowflake

### הכנת הפרמטרים:

In [ ]:
# פרטי חיבור
username = 'YOUR_USERNAME'
password = 'YOUR_PASS'
account = 'your-account.region'  # לדוגמה: abc12345.us-east-1
database = 'POLARS_COOKBOOK_DEMO_YK'
warehouse = 'COMPUTE_WH'          # Warehouse לחישוב
role = 'ACCOUNTADMIN'             # תפקיד המשתמש
schema = 'SANDBOX'
table = 'TITANIC_DATASET'

print("✅ פרמטרי חיבור הוגדרו!")

### 🎓 הסבר על מושגים:

- **Account**: מזהה ייחודי של חשבון Snowflake שלכם
- **Warehouse**: משאב חישוב (כמו CPU/RAM)
- **Database**: קולקציה של schemas
- **Schema**: קולקציה של טבלאות
- **Role**: הרשאות גישה

---

<a id="reading-from-snowflake"></a>
## 📖 קריאה מ-Snowflake

### שיטה 1: ADBC (מומלץ - הכי מהיר)

In [ ]:
# שאילתה
query = f'SELECT * FROM {table}'

# בניית URI
uri = (
    f'snowflake://{username}:{password}@{account}/{database}/{schema}'
    f'?warehouse={warehouse}&role={role}'
)

# קריאה עם ADBC
df = pl.read_database_uri(
    query,
    uri,
    engine='adbc'  # Arrow Database Connectivity - מהיר מאוד!
)

print("✅ נתונים נקראו מ-Snowflake!")
print(f"📊 {df.shape[0]:,} שורות × {df.shape[1]} עמודות")
df.head()

### ⚡ למה ADBC?

**ADBC (Arrow Database Connectivity)** הוא תקן חדש:

- 🚀 **מהיר**: 10-100x מהיר מODBC/JDBC
- 💾 **יעיל**: העברת נתונים ללא המרות
- 🎯 **מודרני**: תוכנן במיוחד עבור נתונים בסקאלה גדולה

---

<a id="alternative-snowflake-methods"></a>
### שיטה 2: Snowflake Connector

In [ ]:
import snowflake.connector

# יצירת חיבור
conn = snowflake.connector.connect(
    user=username,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

# קריאה דרך Arrow
df = pl.from_arrow(
    conn.cursor()
    .execute(query)
    .fetch_arrow_all()
)

print("✅ נתונים נקראו דרך Snowflake Connector!")
df.head()

### שיטה 3: read_database

In [ ]:
# שימוש ישיר בחיבור
df = pl.read_database(query, connection=conn)

print("✅ קריאה עם read_database!")
df.head()

### 📊 השוואת שיטות קריאה:

| שיטה | מהירות | קלות שימוש | מומלץ ל |
|------|--------|------------|--------|
| ADBC | ⚡⚡⚡ | ⭐⭐⭐ | נתונים גדולים |
| Connector + Arrow | ⚡⚡ | ⭐⭐ | שימוש כללי |
| read_database | ⚡ | ⭐⭐⭐ | פשטות |

---

## 💡 טיפים לעבודה עם Snowflake:

### 1. ניהול Warehouses:
```sql
-- השהה warehouse כשלא בשימוש
ALTER WAREHOUSE my_wh SUSPEND;

-- שנה גודל לפי צורך
ALTER WAREHOUSE my_wh SET WAREHOUSE_SIZE = 'LARGE';
```

### 2. אופטימיזציה:
- השתמשו ב-**Clustering Keys** לטבלאות גדולות
- נצלו **Result Caching** - תוצאות נשמרות ל-24 שעות
- השתמשו ב-**Materialized Views** לחישובים חוזרים

### 3. אבטחה:
```python
# אל תשמרו סיסמאות בקוד!
import os
password = os.environ.get('SNOWFLAKE_PASSWORD')
```

---

## 📊 תרגיל 5: Snowflake

### 🎯 משימה:
1. התחבר ל-Snowflake
2. חשב ממוצע גיל לפי נמל עלייה (Embarked)
3. סנן רק נמלים עם 100+ נוסעים

In [ ]:
# כתוב שאילתה ב-SQL
port_analysis_query = """
SELECT 
    ...
-- השלם את השאילתה
"""

# הרץ ב-Snowflake
# ...


---

<a id="exercises"></a>
# 📊 תרגילים מתקדמים

## תרגיל 6: השוואה בין פלטפורמות

### 🎯 משימה:
קרא את אותו dataset מכל הפלטפורמות (S3, Azure, GCS) והשווה ביצועים.

In [ ]:
import time

def benchmark_read(platform_name, read_function):
    """מודד זמן קריאה"""
    start = time.time()
    df = read_function()
    elapsed = time.time() - start
    print(f"{platform_name}: {elapsed:.2f} שניות - {df.shape[0]:,} שורות")
    return elapsed

# השלם את הפונקציות
# benchmark_read("S3", lambda: pl.read_csv(...))
# benchmark_read("Azure", lambda: ...)
# benchmark_read("GCS", lambda: ...)

## תרגיל 7: Pipeline מלא

### 🎯 משימה:
צור pipeline שמעביר נתונים בין פלטפורמות:

1. קרא מ-S3
2. נקה ועבד נתונים ב-Polars
3. שמור ל-BigQuery
4. בדוק את התוצאה

In [ ]:
def data_pipeline():
    """Pipeline מלא להעברת נתונים"""
    
    # שלב 1: קריאה מ-S3
    print("📥 קורא מ-S3...")
    # df = ...
    
    # שלב 2: עיבוד
    print("⚙️ מעבד נתונים...")
    # df_processed = df.filter(...).select(...)
    
    # שלב 3: כתיבה ל-BigQuery
    print("📤 שומר ל-BigQuery...")
    # ...
    
    # שלב 4: אימות
    print("✅ מאמת...")
    # verification = ...
    
    print("🎉 Pipeline הושלם!")
    return verification

# הרץ pipeline
# result = data_pipeline()

---

<a id="summary"></a>
# 🎓 סיכום ומשאבים

## 📝 סיכום הפרק

### מה למדנו:

✅ **Amazon S3**:
- קריאה וכתיבה פשוטה עם `s3://`
- אימות עם AWS credentials
- שימוש ב-s3fs ו-PyArrow

✅ **Azure Blob Storage**:
- עבודה עם `az://` protocol
- שימוש ב-adlfs
- הבדלים בין Blob ל-ADLS

✅ **Google Cloud Storage**:
- קריאה עם `gs://`
- אימות עם Service Account
- שימוש ב-gcsfs

✅ **BigQuery**:
- שאילתות SQL
- קריאה עם ConnectorX/Arrow
- העלאה ישירה

✅ **Snowflake**:
- חיבור עם ADBC
- שיטות קריאה שונות
- אופטימיזציה וחיסכון

---

## 🎯 מתי להשתמש במה?

### פלטפורמת אחסון:

| פלטפורמה | משתמש ב | יתרונות | חסרונות |
|-----------|---------|----------|----------|
| **S3** | AWS | מבוגר, יציב | מורכב לניהול |
| **Azure Blob** | Microsoft | אינטגרציה | מחיר |
| **GCS** | Google | מהיר, פשוט | פחות אופציות |

### מסד נתונים:

| מסד נתונים | משתמש ב | יתרונות | חסרונות |
|------------|---------|----------|----------|
| **BigQuery** | Google | מהיר מאוד | מחיר לשאילתות |
| **Snowflake** | Multi-cloud | גמיש | יקר |

---

## 💡 Best Practices

### 1. אבטחה:
```python
# ✅ טוב - משתני סביבה
import os
api_key = os.environ.get('API_KEY')

# ❌ רע - בקוד
api_key = 'my-secret-key-12345'
```

### 2. ביצועים:
- השתמשו ב-`scan_*` במקום `read_*` לנתונים גדולים
- סננו מוקדם ככל האפשר
- השתמשו ב-Parquet במקום CSV

### 3. עלויות:
- קראו רק מה שצריך
- השתמשו ב-lifecycle policies
- סגרו warehouses/connections כשלא בשימוש

### 4. תחזוקה:
```python
# השתמשו ב-context managers
with fs.open(path, 'wb') as f:
    df.write_parquet(f)
# הקובץ נסגר אוטומטית
```

---

## 📚 משאבים נוספים

### תיעוד רשמי:
- [Polars Docs - Cloud Storage](https://pola-rs.github.io/polars/)
- [AWS S3 Documentation](https://docs.aws.amazon.com/s3/)
- [Azure Blob Docs](https://docs.microsoft.com/azure/storage/blobs/)
- [Google Cloud Storage](https://cloud.google.com/storage/docs)
- [BigQuery Docs](https://cloud.google.com/bigquery/docs)
- [Snowflake Docs](https://docs.snowflake.com/)

### כלים:
- [s3fs](https://s3fs.readthedocs.io/)
- [adlfs](https://github.com/fsspec/adlfs)
- [gcsfs](https://gcsfs.readthedocs.io/)
- [PyArrow](https://arrow.apache.org/docs/python/)
- [ConnectorX](https://github.com/sfu-db/connector-x)

### קהילה:
- [Polars Discord](https://discord.gg/4UfP5cfBE7)
- [Stack Overflow - Polars](https://stackoverflow.com/questions/tagged/python-polars)

---

## 🎉 סיום

**כל הכבוד!** 🎊 סיימת את המדריך המקיף לעבודה עם מקורות נתונים בענן.

### מה הלאה?

1. 💪 **תרגול**: נסה את הקוד עם הנתונים שלך
2. 🔨 **פרויקט**: בנה pipeline אמיתי
3. 📖 **למד עוד**: חקור נושאים מתקדמים
4. 🤝 **שתף**: עזור לאחרים בקהילה

### זוכר:
- ⚡ **התחל קטן** - אל תנסה הכל בבת אחת
- 🔍 **נסה** - ניסוי וטעייה הם חלק מהלמידה
- 💬 **שאל** - הקהילה כאן לעזור
- 🎯 **התמקד** - בחר פלטפורמה אחת ללמוד לעומק

**המדריך המלא שלך ל-Polars ולענן מוכן!** 🚀

---

<div style="text-align: center; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 10px; color: white;">
    <h2>🐻‍❄️ המשך ללמוד ולהתפתח! 🐻‍❄️</h2>
    <p>עם Polars, הענן הוא הגבול!</p>
</div>